In [ ]:
import pandas as pd
import json

In [ ]:
preds = json.load(open('../checkpoints/hoivit_ek_wmotion_doubleatt_ant/val.json','r'))
preds = preds['results']
pred_dict = {}
for key in preds.keys():
    verb_pred_dict = preds[key]['verb']
    noun_pred_dict = preds[key]['noun']
    best_verb_val = max(verb_pred_dict, key=verb_pred_dict.get)
    best_noun_val = max(noun_pred_dict, key=noun_pred_dict.get)
    pred_dict[key] = {'verb': best_verb_val, 'noun': best_noun_val}

In [ ]:
gt = pd.read_csv('../epic-kitchens-100-annotations/EPIC_100_validation.csv')
gt_dict = {}
for row in gt.iterrows():
    gt_dict[row[1]['narration_id']] = {'verb' : row[1]['verb_class'], 'noun' : row[1]['noun_class']}

In [ ]:
correct_verb = [0 for _ in range(97)]
correct_noun = [0 for _ in range(300)]

total_verb = [0 for _ in range(97)]
total_noun = [0 for _ in range(300)]


for key in pred_dict.keys():
    if key in gt_dict.keys():
        if int(pred_dict[key]['verb']) == int(gt_dict[key]['verb']):
            correct_verb[int(pred_dict[key]['verb'])] += 1
        if int(pred_dict[key]['noun']) == int(gt_dict[key]['noun']):
            correct_noun[int(pred_dict[key]['noun'])] += 1

for key in gt_dict.keys():
    total_verb[int(gt_dict[key]['verb'])] += 1
    total_noun[int(gt_dict[key]['noun'])] += 1

In [ ]:
percent_verb_correct = [0 for _ in range(97) ]
percent_noun_correct = [0 for _ in range(300)]
for i, (item1,item2,item3,item4) in enumerate(zip(correct_verb,total_verb,correct_noun,total_noun)):
    if item2:
        percent_verb_correct[i] = (item1/item2)
    if item3:
        percent_noun_correct[i] = (item3/item4)

In [ ]:
import numpy as np

top10_verbs = np.argsort(percent_verb_correct)[-10:]
top10_nouns = np.argsort(percent_noun_correct)[-10:]

actions = pd.read_csv('../epic-kitchens-100-annotations/actions.csv')

In [ ]:
correct_action = np.zeros((97,300))
total_action = np.zeros((97,300))


for key in pred_dict.keys():
    if key in gt_dict.keys():
        if int(pred_dict[key]['verb']) == int(gt_dict[key]['verb']) and \
            int(pred_dict[key]['noun']) == int(gt_dict[key]['noun']):
            correct_action[int(pred_dict[key]['verb'])][int(pred_dict[key]['noun'])] += 1

for key in gt_dict.keys():
    total_action[int(gt_dict[key]['verb'])][int(gt_dict[key]['noun'])] += 1

In [66]:
percent_action_correct = np.divide(correct_action, total_action, out=np.zeros_like(correct_action), where=total_action!=0)
def k_largest_index_argpartition_v1(a, k):
    idx = np.argpartition(-a.ravel(),k)[:k]
    return np.column_stack(np.unravel_index(idx, a.shape))

top10_actions = k_largest_index_argpartition_v1(percent_action_correct,10)

In [69]:
for verb in top10_verbs:
    print(list(actions[actions['verb'] == verb]['action'].values)[0].split()[0])

set
transition
dry
open
put
knead
take
cut
wash
mix


In [70]:
for noun in top10_nouns:
    print(list(actions[actions['noun'] == noun]['action'].values)[0].split()[1])

bin
microwave
tap
plate
cupboard
pizza
fridge
dough
mat
peach


In [68]:
for verb, noun in top10_actions:
    print(list(actions.loc[(actions['verb'] == verb) & (actions['noun'] == noun)]['action'].values))

['mix meat']
['cut pizza']
['dry pan']
['remove onion']
['turn knob']
['remove cover']
['crush spice']
['take parsley']
['close squash']
['pour sugar']
